# Phase 1 

### Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [2]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])
display(all_projects_source_codes.iloc[2])
display(all_projects_bugreports.iloc[0].name)
display(all_projects_source_codes.iloc[2].name)

*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

filename            \gitrepo\src\java\org\apache\commons\collectio...
unprocessed_code    /*\n *  Licensed to the Apache Software Founda...
project                                                   COLLECTIONS
Name: 2, dtype: object

'217'

2

In [3]:
# need to get the file names in the bug report an source code to be the same

# there is no index connection between bugreports index (.name) and the index of the source code
display(all_projects_bugreports.iloc[0].fix)
display(all_projects_source_codes.iloc[217].filename)


array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

'\\gitrepo\\src\\java\\org\\apache\\commons\\collections\\list\\transformedlist.java'

## Removing composite varaibles

In [4]:
import re
#remove next line characters:
def remove_new_lines(text):
#     return str(x).replace('\n', '').replace('*', '').replace('/', '').replace('\\','').replace('\t','')
    text = str(text)
    COMBINE_WHITE_SPACE = re.compile(r"(?a:\s+)")
    text = COMBINE_WHITE_SPACE.sub(' ', text)
    return text.replace('*', '').replace('/', '').replace('\\','')
    
# clean up the various white space and remove some *
def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

# clean up the description and summary, they will both be used for the query
def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|S')
    df.description = df.description.apply(remove_new_lines)
    return df

# need to reformat the source code so it can be compared to the bug reports fix array
def format_sc_filename(x):
  
    x = x.split('apache')
    if len(x) != 2:
        x = x[0].split('springframework')
        if len(x) != 2:
            x = x[0].split('wildfly')
            if len(x) == 2:
                x = 'org.wildfly' + x[1]
        else:
             x = 'org.springframework' + x[1]
    else:
         x = 'org.apache' + x[1]
    if len(x) == 1:
        x = x[0]
   
    x = str(x).replace("\\",".")
    return x

# apply the fixes to the filename 
def clean_sc_filepath(df):
    df.filename = df.filename.apply(format_sc_filename)
    return df



In [5]:
#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df


In [6]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# add the Java key words to the stop words
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
java_operators = ["+", "-", "*", "/", "%", "+=", "-=", "*=", "/=", "++", "--", "==", "!=", "<", ">", "<=", ">=", ".", "[", "]", "(",")", "!", "~","instanceof", "<<", ">>", ">>>", "&", "^", "|", "&&", "||", "?", ":", "^=", "%=", "<<=", ">>=", ">>>=", "&="]
stop = java_keywords + java_operators
STOP_WORDS = ENGLISH_STOP_WORDS.union(stop)
# STOP_WORDS


### Start working on TF-IDF and Cosine similarity calculations

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clean up the unprocessed code column
def clean_source_df(df):
    df = clean_new_lines_source_code(df)
    df = clean_composite_source_code(df)
    df = clean_sc_filepath(df)
    return df

# add the summary and description together and clean the data
def clean_combine_bug_df(df):
    df = clean_new_lines_bug_report(df)
    df = clean_composite_bug_report(df)
    df["query"] = df["summary"] + df["description"]
    return df

# get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_combine_bug_df(all_projects_bugreports)


# get a list of the projects 
print(sc_df.project.unique())
projects = sc_df.project.unique()

# group the data frames
# TODO ignore index here? Then maintain original index and concat the data frames
sc_grouped_df = sc_df.groupby(sc_df.project)
bg_grouped_df = br_df.groupby(br_df.project)

# example of getting a data frame for a single project
col_df = sc_grouped_df.get_group("COLLECTIONS")
display(col_df)




['COLLECTIONS' 'CONFIGURATION' 'IO' 'LANG' 'DATACMNS' 'DATAMONGO'
 'DATAREST' 'LDAP' 'SEC' 'SOCIALFB' 'SPR' 'ELY']


,filename,unprocessed_code,project
0,org.apache.commons.collections.arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,org.apache.commons.collections.bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,org.apache.commons.collections.bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,org.apache.commons.collections.beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,org.apache.commons.collections.bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
471,org.apache.commons.collections.set.testtransfo...,Licensed to the Apache Software Foundation ...,COLLECTIONS
472,org.apache.commons.collections.set.testtypedse...,Licensed to the Apache Software Foundation ...,COLLECTIONS
473,org.apache.commons.collections.set.testtypedso...,Licensed to the Apache Software Foundation ...,COLLECTIONS
474,org.apache.commons.collections.set.testunmodif...,Licensed to the Apache Software Foundation ...,COLLECTIONS


In [58]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_similarity(src_vect, query):
    cosSim = cosine_similarity(query, src_vect).flatten()
    return cosSim

# fit the vectorizer and transform data
def get_similarity(vect, src_vect, query_data):
    query = vect.transform(query_data)
    similarity = calculate_similarity(src_vect,query)
     
    return similarity

# remove the stem and stop words
# takes in an array of strings returns an array of strings
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()
    return text

# take in the source code df for a project and a signle query return scores
def generate_scores_list(source, query):
    scores_list = [] 
    # get a list of strings from the data frames to be vectorized
    source['unprocessed_code'] = source['unprocessed_code'].map(stem_stop)
    query["query"] = query["query"].map(stem_stop)
    query_str = query["query"].iloc[0]
     
    # fit a vectorizer to the data
    vect = TfidfVectorizer(min_df=1).fit(source['unprocessed_code'])
    src_vect = vect.transform(source['unprocessed_code'])
#     display(src_vect)
    
    #get the similarity scores for the bug reports    
    #iterate through the query
    for q in query["query"]:
        similarity = get_similarity(vect, src_vect, [q])
        scores_list.append(similarity)
        # some sort of data structure (dict) to collect the queries to add it to the dataframe later
#         display(similarity)
    return scores_list


def rank_sim_scores(scores):
    sim_scores = list()

#     print(len(scores)) # number of bug reports
    for score in scores:
        indicies = range(len(score))
#         print(len(score)) # number of source code files for this project
        scores_tuple = tuple(zip(score,indicies))
        sorted_tuple = sorted(scores_tuple, reverse = True)
#         display(sorted_tuple)
        sim_scores.append(sorted_tuple)
    
    return sim_scores

# look through the src data frame to find where the fix is. 
def get_fix_indexes(bug, src):
    fix_list = list()
    for fixes in bug["fix"]:
#         print(fixes)
        fix_sub=list()
        for fix in fixes:
#             print(fix)
            df = src[src["filename"].str.match(fix)]
            if(df.shape[0] != 0):
                fix_sub.append(df.index[0])
            else:
                fix_sub.append(-1)
#         print(fix_sub)
        fix_list.append(fix_sub)
#     print(fix_list)
    # this is a list of the indexes of the file where the fix was located
    return fix_list
        
        


    
def generate_all_scores():
    
    all_bugs = []
    all_src = []
    # iterate through the list of 12 projects
    for proj in projects:
        # create dataframes for each project
        src_df = sc_grouped_df.get_group(proj)
        bug_df = bg_grouped_df.get_group(proj).copy()
        
        # generate the scores list
        scores = generate_scores_list(src_df, bug_df)
    
        #append scores list to the bug dataframe
        bug_df["sim_vect"] = scores # the only way that the matrix is related to the src code 
                                    # is through the index.
        
        # Rank the source code files and know what file(s) the bug was in
        bug_df["ranked_sim"] = rank_sim_scores(scores)
        display(bug_df["ranked_sim"])
        # get list of where the fixes rank
        bug_df["fix_indexes"] = get_fix_indexes(bug_df, src_df)
        display(bug_df)
        #get location of the fix indexes within the ranked_sim
        
        # CALCULATE THE MAP AND MRR HERE WITH A FUNCTION AND ADD IT TO THE BUGS DATAFRAME
    
        
        # maintain a list of all the dataframes
        all_bugs.append(bug_df)
        all_src.append(src_df)
    # concatenate all the data frames in order    
    all_bug_df = pd.concat(all_bugs, ignore_index=True)
    all_src_df = pd.concat(all_src, ignore_index=True)
    return all_bug_df, all_src_df

## TODO list
- Figure out index of the top 20 in the bug sim_vect
- Figure out index of files where the fix was ( this is in the bug report )
- Calculate MAP and MRR for each
- Generate graphs and report on the findings

In [59]:
bugs, sources = generate_all_scores()

C:\Users\kfrie\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


id
217    [(0.4656718955142169, 222), (0.447279509685931...
214    [(0.10345136646680671, 3), (0.0880717932156749...
222    [(0.38644080002864695, 288), (0.37509355439192...
261    [(0.5591516607470604, 305), (0.503968358691063...
264    [(0.6582236412487737, 298), (0.643831204240897...
                             ...                        
577    [(0.12069235153633603, 109), (0.10860615800701...
580    [(0.23762779658622352, 137), (0.22792879438408...
583    [(0.0725756081794044, 60), (0.0442830912391032...
586    [(0.33840731012035485, 427), (0.32327573122239...
597    [(0.2173144095633128, 303), (0.194166721215988...
Name: ranked_sim, Length: 92, dtype: object

,fix,text,fixdate,summary,description,project,average_precision,query,sim_vect,ranked_sim,fix_indexes
id,,,,,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3 Map. Entry.set Value() overwrites other ...,Flat3 Map&amp;apos;s Entry objects will overw...,COLLECTIONS,0.0,flat3 map. entry.set value() overwrit entri va...,"[0.058548134080555064, 0.08970216513375899, 0....","[(0.4656718955142169, 222), (0.447279509685931...","[233, 436]"
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,Extended Properties - field include should be ...,"The field ""include"" in Extended Properties is...",COLLECTIONS,0.0,extend properti field includ non-stat privatet...,"[0.013388795565578315, 0.028400683146419893, 0...","[(0.10345136646680671, 3), (0.0880717932156749...","[292, 22]"
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,Collection Utils remove All is actually retain...,"The remove All( Collection collection, Collec...",COLLECTIONS,0.0,collect util remov all actual retain allth rem...,"[0.04851850840676917, 0.16573102168998424, 0.0...","[(0.38644080002864695, 288), (0.37509355439192...","[303, 288, 15]"
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3 Map.remove() does not return the correct...,Flat3 Map m = new Flat3 Map();n m.put( new I...,COLLECTIONS,0.0,flat3 map.remove() doe correct valu size 3flat...,"[0.03731537102786185, 0.046074298711050336, 0....","[(0.5591516607470604, 305), (0.503968358691063...","[233, 436]"
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,Fast Tree Map forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0,fast tree map forget comparatorin line 359 582...,"[0.016509059582201484, 0.016813266781609673, 0...","[(0.6582236412487737, 298), (0.643831204240897...",[27]
...,...,...,...,...,...,...,...,...,...,...,...
577,[org.apache.commons.collections4.trie.patricia...,NaN,2015-09-28 09:45:19,Patricia Trie bugs when only a few bits change,"I have a bug report for you, for the class Ab...",COLLECTIONS,0.0,"patricia trie bug bit changei bug report you, ...","[0.0264495015828069, 0.06419356614392692, 0.00...","[(0.12069235153633603, 109), (0.10860615800701...",[-1]
580,[org.apache.commons.collections4.multimap.mult...,NaN,2015-11-21 10:02:58,Arbitrary remote code execution with Invoker ...,With Invoker Transformer serializable collect...,COLLECTIONS,0.0,arbitrari remot code execut invok transformerw...,"[0.05176015827563829, 0.11265190941103775, 0.0...","[(0.23762779658622352, 137), (0.22792879438408...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
583,[org.apache.commons.collections4.multimap.mult...,NaN,2015-12-14 07:47:22,"J A V A serialization vulnerability "" C V E-20...",The vulnerability we are talking about is know...,COLLECTIONS,0.0,"J A V A serial vulner "" C V e-2015-4852"" commo...","[0.018726170213306553, 0.025489324635428728, 0...","[(0.0725756081794044, 60), (0.0442830912391032...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


KeyboardInterrupt: 

In [12]:
def findBugRanks(bug_vectors, source_codes):
    
    similarity_scores = []
    rank_indexs = []
    
    for project, vect, fixes in zip(bug_vectors['project'],bug_vectors['sim_vect'],bug_vectors['fix']): 
        
        code = sc_grouped_df.get_group(project)
        code = code['filename'].to_numpy()  ## find all code files pertaining to the bug reports project 
        
        z = [code for _,code in sorted(zip(vect,code),reverse=True)] ## sort the list of projects based on the similarity scores vector
        sorted_array = np.sort(vect)[::-1]
            
        for fix in fixes:
            try:
                index = z.index(fix)
                score = sorted_array[index]
                similarity_scores.append(score)
                rank_indexs.append(index)
                
                

            except:
                similarity_scores.append(fix)
                rank_indexs.append(-1)
                
    tup = zip(similarity_scores,rank_indexs) # zip the similarity scores along with the rank 
    
    return tup


tup = findBugRanks(bugs, sc_grouped_df)
tuple(tup)

((0.4472795096859311, 1),
 (0.2913655231050407, 25),
 (0.03209980879562304, 16),
 (0.0880717932156749, 1),
 (0.31912456699257075, 4),
 (0.38644080002864695, 0),
 (0.28046076214130816, 8),
 (0.21328436467544076, 29),
 (0.5039683586910633, 1),
 (0.6438312042408976, 1),
 (0.423608812618834, 1),
 (0.5026370609304508, 0),
 (0.21673040369631835, 16),
 (0.43031369493789906, 1),
 (0.4909623398612745, 0),
 (0.2512620370028016, 14),
 (0.4730434626385859, 1),
 (0.486343809828954, 0),
 (0.13743009617270954, 51),
 (0.36345491909775834, 91),
 (0.40301763571397675, 1),
 (0.501993096040347, 0),
 (0.3028765649566887, 7),
 ('org.apache.commons.collections.splitmap.abstractiterablegetmapdecorator.java',
  -1),
 ('org.apache.commons.collections.splitmap.testtransformedmap.java', -1),
 (0.14718196728468272, 10),
 (0.23501428756066586, 1),
 (0.17707827762462128, 5),
 (0.29015004202166644, 21),
 (0.34800929761560623, 2),
 (0.4398117899012724, 0),
 (0.2478632221663548, 7),
 (0.32635182109160366, 2),
 (0.35617

In [47]:
src = sc_grouped_df.get_group("COLLECTIONS")
bug = bg_grouped_df.get_group("COLLECTIONS")
count = 0
fix_list = list()
for fixes in bug["fix"]:
    print(fixes)
    fix_sub=list()
    for fix in fixes:
        print(fix)
        df = src[src["filename"].str.match(fix)]
        display(df)
        try:
            print(df.index[0])
            fix_sub.append(df.index[0])
        except:
            count += 1
            fix_sub.append(-1)
    print(fix_sub)
    fix_list.append(fix_sub)
print(fix_list)
print(count) # 328 missing
# this is a list of the indexes of the file where the fix was located


,fix,text,fixdate,summary,description,project,average_precision,query
id,,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3 Map. Entry.set Value() overwrites other ...,Flat3 Map&amp;apos;s Entry objects will overw...,COLLECTIONS,0.0,Flat3 Map. Entry.set Value() overwrites other ...
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,Extended Properties - field include should be ...,"The field ""include"" in Extended Properties is...",COLLECTIONS,0.0,Extended Properties - field include should be ...
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,Collection Utils remove All is actually retain...,"The remove All( Collection collection, Collec...",COLLECTIONS,0.0,Collection Utils remove All is actually retain...
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3 Map.remove() does not return the correct...,Flat3 Map m = new Flat3 Map();n m.put( new I...,COLLECTIONS,0.0,Flat3 Map.remove() does not return the correct...
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,Fast Tree Map forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0,Fast Tree Map forgets the comparatorIn line 35...
...,...,...,...,...,...,...,...,...
577,[org.apache.commons.collections4.trie.patricia...,NaN,2015-09-28 09:45:19,Patricia Trie bugs when only a few bits change,"I have a bug report for you, for the class Ab...",COLLECTIONS,0.0,Patricia Trie bugs when only a few bits change...
580,[org.apache.commons.collections4.multimap.mult...,NaN,2015-11-21 10:02:58,Arbitrary remote code execution with Invoker ...,With Invoker Transformer serializable collect...,COLLECTIONS,0.0,Arbitrary remote code execution with Invoker ...
583,[org.apache.commons.collections4.multimap.mult...,NaN,2015-12-14 07:47:22,"J A V A serialization vulnerability "" C V E-20...",The vulnerability we are talking about is know...,COLLECTIONS,0.0,"J A V A serialization vulnerability "" C V E-20..."


['org.apache.commons.collections.map.flat3map.java'
 'org.apache.commons.collections.map.testflat3map.java']
org.apache.commons.collections.map.flat3map.java


,filename,unprocessed_code,project
233,org.apache.commons.collections.map.flat3map.java,Licensed to the Apache Software Foundation ...,COLLECTIONS


233
org.apache.commons.collections.map.testflat3map.java


,filename,unprocessed_code,project
436,org.apache.commons.collections.map.testflat3ma...,Licensed to the Apache Software Foundation ...,COLLECTIONS


...

org.apache.commons.collections4.trie.patriciatrietest.java


,filename,unprocessed_code,project


[-1, -1]
['org.apache.commons.collections4.functors.catchandrethrowclosure.java']
org.apache.commons.collections4.functors.catchandrethrowclosure.java


,filename,unprocessed_code,project


[-1]
[[233, 436], [292, 22], [303, 288, 15], [233, 436], [27], [303, 288, 15], [303, 288, 15], [303, 288, 15], [223], [303, 288, 15], [-1, -1], [303, 288, 15], [387, 178], [169, 385, 108], [235, 221, 228, 223, 437, 438, 234, 431, 441, 238], [36], [303, 288, 15], [152, 24], [57], [15], [36], [303, 33], [261], [261], [77], [15], [206], [303, 288, 15], [303, 288, 15], [303, 288, 15], [223], [223], [303, 33], [215, 420], [33], [431, 228], [292, 22], [203, 409], [215, 420], [242, 446], [325, 313, 71, 58], [240], [215, 420], [292, 22], [36], [432, 461, 100, 229, 354], [350, 96, 273], [242, 446], [303, 288, 15], [113, -1, 16], [-1, -1, -1, 100, 260], [-1, -1, -1, 215], [-1, -1], [323, 51, 470, 12, 70, 69, 217, 33, 349, 103, 471, 268, 94, 422, 15, 2, 267, 357, 324], [-1], [-1, -1, -1, -1, -1, -1], [-1], [-1, -1, -1, -1], [-1, -1], [-1, -1], [303, 288, 15], [-1, -1], [-1, -1], [-1, -1, -1], [-1, -1], [-1, -1, -1], [-1, -1], [-1], [-1], [-1, -1], [-1], [-1], [-1], [-1], [-1, -1, -1, -1, -1, -1, 